In [10]:
import csv
import numpy as np
import pandas as pd

In [11]:
# reading in and preprocessing the train dataset

train_dataset = []

with open("C:/Users/katri/Downloads/bbc_newstopics/bbc_train.csv", encoding="utf-8") as f:
    rd = csv.reader(f)
    for topic, text in rd:
        words = [w.lower() for w in text.split() if len(w) > 3] # removes tokens < 3 characters and lowercases all tokens
        article = [topic, words]
        train_dataset.append(article)

print(train_dataset[0])

['sport', ['henry', 'tipped', 'fifa', 'award', 'fifa', 'president', 'sepp', 'blatter', 'hopes', 'arsenal', 'thierry', 'henry', 'will', 'named', 'world', 'player', 'year', 'monday', 'henry', 'fifa', 'shortlist', 'with', 'barcelona', 'ronaldinho', 'newly', 'crowned', 'european', 'footballer', 'year', 'milan', 'andriy', 'shevchenko', 'blatter', 'said', 'henry', 'personality', 'field', 'organise', 'game', 'winner', 'accolade', 'will', 'named', 'glittering', 'ceremony', 'zurich', 'opera', 'house', 'three', 'shortlisted', 'candidates', 'women', 'award', 'hamm', 'united', 'states', 'germany', 'birgit', 'prinz', 'brazilian', 'youngster', 'marta', 'hamm', 'recently', 'retired', 'looking', 'regain', 'women', 'award', 'which', 'lost', 'last', 'year', 'striker', 'prinz', 'fifa', 'changed', 'panel', 'voters', 'this', 'year', 'awards', 'male', 'female', 'captains', 'every', 'national', 'team', 'will', 'able', 'vote', 'well', 'their', 'coaches', 'fipro', 'global', 'organisation', 'professional', 'pla

In [12]:
# converting into pandas dataframe for better readability

df_train_dataset = pd.DataFrame(train_dataset)
df_train_dataset.head(5)

,0,1
0,sport,"[henry, tipped, fifa, award, fifa, president, ..."
1,tech,"[peer, peer, nets, here, stay, peer, peer, net..."
2,tech,"[global, blogger, action, called, global, blog..."
3,business,"[beer, giant, swallows, russian, firm, brewing..."
4,business,"[buyers, snap, airways, shares, investors, hav..."


In [13]:
topics = ['tech', 'politics', 'business', 'entertainment', 'sport']

In [1]:
# creating unique vocabulary from the train dataset

vocabulary = []

for article in train_dataset:
    words_list = article[1]
    for word in words_list:
        vocabulary.append(word)

unique_vocabulary = list(set(vocabulary)) # converting list into set removes the duplicates
        
print(unique_vocabulary[0:10])

NameError: name 'train_dataset' is not defined

In [15]:
length_vocabulary = len(vocabulary)
print (f"Total vocabulary: {length_vocabulary}")

length_unique_vocabulary = len(unique_vocabulary)
print (f"Unique vocabulary (V): {length_unique_vocabulary}")

Total vocabulary: 499654
Unique vocabulary (V): 27308


In [16]:
# calculating a probability that the word w is present in articles with the topic is c, 
# according to the formula P(w|c)= (N_w,c+1)/N_c+|V|), where:
# N_w,c (occurrencies) - number of occurrences of word w in articles with the topic c
# N_c (total_count) - total number of words in articles with the topic c
# |V| (length_unique_vocabulary) - number of unique words over all articles and topics

In [17]:
# occurrencies of each unique word in the articles with each of the topics

occurrencies = []

for topic in topics:
    for word in unique_vocabulary:
        count = 0
        for article in train_dataset:
            if article[0] == topic:
                words_list = article[1]
                frequency = words_list.count(word) # counts unique words
                count += frequency
        row = [topic, word, count]
        occurrencies.append(row)

print(occurrencies[0:10])

[['tech', 'hefty', 3], ['tech', 'venezuelan', 0], ['tech', 'renny', 0], ['tech', 'eliminates', 2], ['tech', 'cough', 0], ['tech', 'aura', 1], ['tech', 'connectors', 3], ['tech', 'pctvts', 1], ['tech', 'choreographer', 0], ['tech', 'heartfelt', 0]]


In [18]:
# presenting occurrencies as pandas dataframe for better readability

df_occ = pd.DataFrame(occurrencies, columns=['Topic', 'Word', 'Occurrencies'])
df_occ.head(5)

,Topic,Word,Occurrencies
0,tech,hefty,3
1,tech,venezuelan,0
2,tech,renny,0
3,tech,eliminates,2
4,tech,cough,0


In [19]:
# total count of words in the articles with each of the topics

total_count = []

for topic in topics:
    count = 0
    for article in train_dataset:
        if article[0] == topic:
            words_list = article[1]
            number_words = len(words_list) # count of words in the article
            count += number_words
    row = [topic, count]
    total_count.append(row)

print(total_count)

[['tech', 118690], ['politics', 109121], ['business', 99636], ['entertainment', 74168], ['sport', 98039]]


In [20]:
# converting the list of word counts into a dictionary for easier access

dict_count = dict(total_count)
print(dict_count)

{'tech': 118690, 'politics': 109121, 'business': 99636, 'entertainment': 74168, 'sport': 98039}


In [21]:
# calculating probabilities for each word

p = []
for index, row in df_occ.iterrows():
    p.append((row['Occurrencies'] + 1)/(dict_count.get(row['Topic']) + length_unique_vocabulary))
df_occ['Probabilities'] = p
df_occ.head(5)

,Topic,Word,Occurrencies,Probabilities
0,tech,hefty,3,0.000027
1,tech,venezuelan,0,0.000007
2,tech,renny,0,0.000007
3,tech,eliminates,2,0.000021
4,tech,cough,0,0.000007


In [22]:
# since we will operate with logs of probabilities, we can caclulate and add them to the df now

log_p = []
for index, row in df_occ.iterrows():
    log_p.append(np.log(row['Probabilities']))
df_occ['Log Probabilities'] = log_p
df_occ.head(5)

,Topic,Word,Occurrencies,Probabilities,Log Probabilities
0,tech,hefty,3,0.000027,-10.505054
1,tech,venezuelan,0,0.000007,-11.891348
2,tech,renny,0,0.000007,-11.891348
3,tech,eliminates,2,0.000021,-10.792736
4,tech,cough,0,0.000007,-11.891348


In [23]:
# calculating number of articles with each topic

count_articles = []
for topic in topics:
    count = 0
    for article in train_dataset:
        if article[0] == topic:
            count+=1
    row = [topic, count]
    count_articles.append(row)
print(count_articles)

[['tech', 380], ['politics', 395], ['business', 488], ['entertainment', 368], ['sport', 494]]


In [24]:
# calculating probabilitises that the topic of the article is c (P(c)) and calculating logs of probabilities

p_c_list = []
for i in count_articles:
    p_c = np.log(i[1]/len(train_dataset))
    row = [i[0], p_c]
    p_c_list.append(row)
print(p_c_list)

[['tech', -1.7213558286380857], ['politics', -1.6826413164573952], ['business', -1.4712116755053701], ['entertainment', -1.7534441431895862], ['sport', -1.4589915641705946]]


In [25]:
# converting list of probabilitises into dictionary for easier access

p_c_dict = dict(p_c_list)
print(p_c_dict)

{'tech': -1.7213558286380857, 'politics': -1.6826413164573952, 'business': -1.4712116755053701, 'entertainment': -1.7534441431895862, 'sport': -1.4589915641705946}


In [26]:
# reading in and preprocessing the test dataset

test_dataset = []

with open("C:/Users/katri/Downloads/bbc_newstopics/bbc_test.csv", encoding="utf-8") as f:
    rd = csv.reader(f)
    for topic, text in rd:
        words = [w.lower() for w in text.split() if len(w) > 3] # removes tokens < 3 characters and lowercases all tokens
        article = [topic, words]
        test_dataset.append(article)
print(test_dataset[0])

['tech', ['apple', 'makes', 'blogs', 'reveal', 'sources', 'apple', 'legal', 'fight', 'make', 'three', 'bloggers', 'reveal', 'told', 'them', 'about', 'unreleased', 'products', 'unmask', 'employees', 'leaking', 'information', 'launched', 'december', '2004', 'following', 'online', 'articles', 'about', 'apple', 'asteroid', 'product', 'apple', 'right', 'mail', 'records', 'from', 'three', 'bloggers', 'root', 'culprit', 'lawyer', 'three', 'bloggers', 'said', 'ruling', 'dangerous', 'precedent', 'that', 'could', 'harm', 'news', 'reporters', 'apple', 'lawsuit', 'accused', 'anonymous', 'people', 'stealing', 'trade', 'secrets', 'about', 'asteroid', 'music', 'product', 'leaking', 'them', 'powerpage', 'apple', 'insider', 'think', 'secret', 'websites', 'three', 'apple', 'sites', 'that', 'obsessively', 'watch', 'iconic', 'firm', 'information', 'about', 'future', 'products', 'apple', 'notoriously', 'secretive', 'about', 'upcoming', 'products', 'which', 'gives', 'snippets', 'information', 'about', 'what

In [27]:
# converting test dataset into pandas dataframe for better readability

df_test_dataset = pd.DataFrame(test_dataset)
df_test_dataset.head(5)

,0,1
0,tech,"[apple, makes, blogs, reveal, sources, apple, ..."
1,politics,"[talks, avert, pension, strike, talks, aimed, ..."
2,business,"[unveils, more, budget, south, african, govern..."
3,entertainment,"[shark, tale, best, seller, oscar, nominated, ..."
4,tech,"[sequel, criminally, good, grand, theft, auto,..."


In [28]:
# predicting the article topic

predicted_probabilities = []

for article in test_dataset:
    probabilities = []
    words_list = article[1]
    for key in p_c_dict:
        topic = key
        sum_log_p = 0 # initiates the sum of word probabilities log(P(w|c)) for each article
        for word in words_list:
        # finds the matching row in the dataframe with word occurrencies - there will be only one for each topic
            matching_row = df_occ[(df_occ['Word'] == word) & (df_occ['Topic'] == topic)]
            if not matching_row.empty:
                # if the matching row is found, calculates the probability
                probability = matching_row['Log Probabilities'].values[0]
                sum_log_p += probability
            else:
                # if there is no matching row, i. e. the word from the test dataset is not present in the train dataset,
                # we use the formula 1/(Nc+|V|)
                probability = np.log(1/(dict_count.get(topic) + length_unique_vocabulary))
                sum_log_p += probability
        # calculates log of probability that the topic of the article is c
        log_h_c = [topic, (p_c_dict.get(topic) + sum_log_p)]
        probabilities.append(log_h_c) # probabilities of all topics for each article
    dict_probabilities = dict(probabilities)
    max_prob = max(zip(dict_probabilities.values(), dict_probabilities.keys()))[1] # finds the most probable topic
    predicted_probabilities.append([article[1], article[0], max_prob])
print(predicted_probabilities)

[[['apple', 'makes', 'blogs', 'reveal', 'sources', 'apple', 'legal', 'fight', 'make', 'three', 'bloggers', 'reveal', 'told', 'them', 'about', 'unreleased', 'products', 'unmask', 'employees', 'leaking', 'information', 'launched', 'december', '2004', 'following', 'online', 'articles', 'about', 'apple', 'asteroid', 'product', 'apple', 'right', 'mail', 'records', 'from', 'three', 'bloggers', 'root', 'culprit', 'lawyer', 'three', 'bloggers', 'said', 'ruling', 'dangerous', 'precedent', 'that', 'could', 'harm', 'news', 'reporters', 'apple', 'lawsuit', 'accused', 'anonymous', 'people', 'stealing', 'trade', 'secrets', 'about', 'asteroid', 'music', 'product', 'leaking', 'them', 'powerpage', 'apple', 'insider', 'think', 'secret', 'websites', 'three', 'apple', 'sites', 'that', 'obsessively', 'watch', 'iconic', 'firm', 'information', 'about', 'future', 'products', 'apple', 'notoriously', 'secretive', 'about', 'upcoming', 'products', 'which', 'gives', 'snippets', 'information', 'about', 'what', 'wor

In [69]:
df_predicted_topics = pd.DataFrame(predicted_probabilities, columns = ['Article', 'Topic', 'Predicted Topic'])
df_predicted_topics.head(100)

,Article,Topic,Predicted Topic
0,"[apple, makes, blogs, reveal, sources, apple, ...",tech,tech
1,"[talks, avert, pension, strike, talks, aimed, ...",politics,politics
2,"[unveils, more, budget, south, african, govern...",business,business
3,"[shark, tale, best, seller, oscar, nominated, ...",entertainment,entertainment
4,"[sequel, criminally, good, grand, theft, auto,...",tech,tech
...,...,...,...
95,"[helps, drive, democracy, asia, kyrgyz, republ...",tech,politics
96,"[real, talks, over, gravesen, move, real, madr...",sport,sport
97,"[hoffman, hits, over, modern, film, hollywood,...",entertainment,entertainment
98,"[giving, financial, gifts, children, your, chi...",business,business


In [71]:
df_predicted_topics['Accuracy'] = df_predicted_topics['Topic'] == (df_predicted_topics['Predicted Topic'])
df_predicted_topics.head(100)

,Article,Topic,Predicted Topic,Accuracy
0,"[apple, makes, blogs, reveal, sources, apple, ...",tech,tech,True
1,"[talks, avert, pension, strike, talks, aimed, ...",politics,politics,True
2,"[unveils, more, budget, south, african, govern...",business,business,True
3,"[shark, tale, best, seller, oscar, nominated, ...",entertainment,entertainment,True
4,"[sequel, criminally, good, grand, theft, auto,...",tech,tech,True
...,...,...,...,...
95,"[helps, drive, democracy, asia, kyrgyz, republ...",tech,politics,False
96,"[real, talks, over, gravesen, move, real, madr...",sport,sport,True
97,"[hoffman, hits, over, modern, film, hollywood,...",entertainment,entertainment,True
98,"[giving, financial, gifts, children, your, chi...",business,business,True


In [74]:
# calculates the error rate of the model

num_false = (df_predicted_topics['Accuracy'] == False).sum()
print (f"Error rate: {num_false}%")

Error rate: 4%
